# Chargement des données et séparation des features et target

Avant tout préprocessing, on établit la séparation des données de train et de test afin de limiter la possibilité de data leak entre les jeux de données de train et de test. Nous avons choisi le ratio standard suivant : 80% de train, 20% de test.

On sait que la classe 2583 (accessoires de piscines) est sur représentée dans le jeu de données. Nous avons décidé de :
- faire de l'undersampling aléatoire pour diminuer sa présence à 3% au lieu de 12%
- faire de l'oversampling aléatoire à 3% pour les classes minoritaires

Cela permet d'équilibrer les classes présentes et améliorer les performances de l'apprentissage.

Le jeu de test lui reste inchangé et déséquilibré comme dans la réalité.

#### Proportion de chaque classe dans les jeux de données

| Classe | Train avant resampling | Train après resampling | Test      |
|--------|------------------------|------------------------|-----------|
| 2583   | 12.022316              | 3.703704               | 12.023081 |
| 1560   | 5.973621               | 3.703704               | 5.976213  |
| 1300   | 5.941235               | 3.703704               | 5.940886  |
| 2060   | 5.879409               | 3.703704               | 5.882007  |
| 2522   | 5.874993               | 3.703704               | 5.876119  |
| 1280   | 5.735147               | 3.703704               | 5.734809  |
| 2403   | 5.621798               | 3.703704               | 5.622939  |
| 2280   | 5.605606               | 3.703704               | 5.605276  |
| 1920   | 5.066832               | 3.703704               | 5.069477  |
| 1160   | 4.654655               | 3.703704               | 4.657325  |
| 1320   | 3.817052               | 3.703704               | 3.815356  |
| 10     | 3.669846               | 3.703704               | 3.668158  |
| 2705   | 3.251781               | 3.703704               | 3.250118  |
| 1140   | 3.145793               | 3.703704               | 3.144136  |
| 2582   | 3.048637               | 3.703704               | 3.049929  |
| 40     | 2.952953               | 3.703704               | 2.955723  |
| 2585   | 2.939704               | 3.703704               | 2.938059  |
| 1302   | 2.933816               | 3.703704               | 2.932171  |
| 1281   | 2.437732               | 3.703704               | 2.437588  |
| 50     | 1.979921               | 3.703704               | 1.978333  |
| 2462   | 1.673733               | 3.703704               | 1.672162  |
| 2905   | 1.027498               | 3.703704               | 1.024494  |
| 60     | 0.980392               | 3.703704               | 0.977390  |
| 2220   | 0.970088               | 3.703704               | 0.971503  |
| 1301   | 0.950951               | 3.703704               | 0.947951  |
| 1940   | 0.945063               | 3.703704               | 0.947951  |
| 1180   | 0.899429               | 3.703704               | 0.900848  |

# Preprocessing des données

## Preprocessing des données textuelles

### Création de la variable `has_description` basée sur la présence de la variable `description`

Nous avons vu lors de l'exploration que la répartition de la présence de description n'est pas homogène parmi les catégories de produits. On peut en déduire que l'absence ou présence de description est une information à part entière qu'il peut être intéressant d'exploiter dans nos modèles.

Nous décidons de créer la variable `has_description` qui vaut :
- 1 si la description est présente
- 0 sinon

### Fusion des variables `designation` et `description` dans `full_description`

La `description` et la `designation` sont toutes les deux des valeurs textuelles libres. Bien que `designation` soit plus courte, elles apportent le même type d'information sur le produit c'est-à-dire sa description et son appellation commune. `description` pouvant être nulle, cela peut être gênant pour son intégration dans les modèles. 

Afin de faciliter le traitement de ces deux valeurs textuelles, nous décidons de les fusionner dans une variable nommée `full_description`

### Nettoyage, tokenisation et stemmatisation de la variable `full_description`

Nous souhaitons faire les étapes de préprocessing suivantes :

- Suppression des stop words (anglais et français)
- Suppression des ponctuations
- Suppression des balises HTML
- Stemmatisation
- Application d'un TF IDF

Après quelques tentatives de création d'un process, nous décidons d'abandonner la stemmatisation. On note avec l'extrait ci-dessous que la stemmatisation est trop violente sur le contenu de la variable d'une part et qu'il n'est pas adapté sur un contenu multilingue tel que nous avons ici (français, anglais mais aussi parfois allemand). Afin de limiter la perte en information, nous décidons de ne pas appliquer de lemmatisation ni de stemmatisation.

|                               |                      preprocessed_full_description|                                   full_description|
|------------------------------:|--------------------------------------------------:|---------------------------------------------------|
|                             0 |                              ship focus record 31 |                          Ships In Focus Record 31 |
|                             1 | convergent divergent alignment standardsassess... | Convergence Or Divergence: Alignment Of Standa... |
|                             2 | lecon scienc cour superieur premier anne certi... | Lecons De Sciences - Cours Superieur - Premier... |
|                             3 |                             gener women historian |                   Generations Of Women Historians |
|                             4 | magnet encycloped spiritual trait spécial fait... | Magnétisme: Encyclopédie Magnétique Spirituali... |
|                             5 | inglorious art peac exhibit canadian society n... | The Inglorious Arts Of Peace: Exhibitions In C... |
|                             6 |              sylvi souvient-il vi giacomo leopard | Sylvia Te Souvient-Il ? - Vie De Giacomo Leopardi |
|                             7 |                             trachom malad pauvret |          Le Trachome - Une Maladie De La Pauvreté |
|                             8 |                   business associ 2006 statut rul |    Business Associations: 2006 Statutes And Rules |
|                             9 |                        luun tom 8 l'attrapeur rêv |               Luuna Tome 8 - L'attrapeur De Rêves |

Il aurait aussi été possible de traduire les textes pour uniformiser la langue employée mais cela représente :
- un traitement coûteux
- un risque de perte d'information si la traduction n'est pas de bonne qualité

Comme le jeu de données est conséquent, on peut s'attendre à ce que les algorithmes d'analyse textuelle arrivent à extraire l'information utile malgré la présence de multiple langues.

Nous décidons donc de limiter le prétraitement textuel aux éléments suivants :
- nettoyage des stopwords anglais et français
- nettoyage de la ponctuation
- nettoyage des balises HTML
- tokenisation des chaînes de caractères

### Suppression des variables à faible variance

Nous décidons de supprimer les variables `productid` et `imageid` car elle n'apportent pas d'information utile pour notre classification et ne sont liées à aucune variance sur la variable cible.

### Application de TF-IDF à la variable `full_description`

En l'état, les chaînes de caractères dans full_description sont difficilement exploitables par des algorithmes de machine learning.

Il est nécessaire de les transformer en valeurs numériques interprétables. Nous avons décidé d'appliquer le TF-IDF pour une extraction de donnée rapide et efficace suite à notre analyse en wordcloud qui fait apparaître la répétition de mots clés pour chaque catégorie de produits.

Nous avons choisi d'appliquer un TF IDF avec :
- 10 000 features
- un n-gram de (1,1)
- un seuil minimal de présence des mots dans 10 documents

Après analyse de la taille des données prétraitées et de leur nombre de dimensions, nous avons décidé de réduire les dimensions via TruncatedSVD. La littérature nous a permis de choisir un seuil acceptable de 200 components. Cette réduction nous permet de nous adapter à notre capacité de calcul limitée sur nos postes personnels.

### Sauvegarde des données textuelles suite à leur préprocessing

Afin de faciliter le travail sur plusieurs modèles et gagner du temps, on décide de sauvegarder les données prétraitées avec joblib au format pkl.

## Preprocessing des images

### Transformation des images




Toutes les images initiales étaient en couleur et de taille 500 × 500 pixels. Afin de réduire la complexité computationnelle et de faciliter l’entraînement des modèles, elles ont été transformées en images en nuances de gris et redimensionnées en 200 × 200 pixels.

Justification :

- La réduction de taille permet de diminuer significativement la dimension des données (de 500 × 500 × 3 ≈ 750 000 valeurs par image à 200 × 200 × 1 = 40 000 valeurs par image).

- Le passage en nuances de gris supprime l’information de couleur jugée moins importante que les formes, motifs et textures dans notre compromis d'efficacité.

Conséquences :

- Perte d’information visuelle : perte de détails lors de la transformation de dimension; et certaines caractéristiques basées sur la couleur disparaissent, ce qui peut limiter la performance si la couleur est discriminante pour certaines classes.

- En contrepartie, cela réduit fortement le coût mémoire et le temps de calcul.

Toutes les images ainsi transformées ont été sauvegardées dans un répertoire commun pour être rélié aux données et utiliser facilement à partir d'un chemin d'accès standardisé.

### Alignement avec les données textuelles (équilibrage des classes)


Pour garantir une cohérence avec les travaux menés sur les données textuelles, la même stratégie d’oversampling et d’undersampling a été appliquée sur les images. Cela permet de sélectionner uniquement les images correspondant aux échantillons retenus après rééquilibrage des classes.

Justification :

- L’alignement entre données textuelles et images est essentiel afin que les modèles multimodaux ou les comparaisons de performances restent pertinents.

- Le rééquilibrage assure une meilleure représentativité des classes minoritaires et évite que les modèles ne soient biaisés vers les classes majoritaires.

Conséquences :

- L’équilibrage artificiel peut introduire une légère distorsion par rapport à la distribution réelle des données, mais il améliore la robustesse du modèle en apprentissage.

### Encodage des classes cibles

Le même encodeur que pour les données textuelles a été utilisé sur les classes. Notamment pour la regréssion logistique, si les classes sont d'ordre numérique, il est plus pertinent de les classer de 0 au nombre de classes (-1).

Justification :

- Plus facilement exploitable par les modèles.

- Cela permet également une comparaison directe des résultats aux modèles textuels..

### Découpage du jeu de données

La séparation en ensemble d’entraînement et ensemble de test a été réalisée selon les mêmes proportions que pour le texte. On utilise le même random_state ce qui permet d'obtenir exactement les mêmes données cibles.

Justification :

- Le respect des mêmes règles de séparation garantit la validité des comparaisons.

- Un jeu de données d'entraînement et de test désignant les mêmes données permet une interprétation plus juste de la pertinence d'un modèle ou d'un autre.

### Réduction de dimension par PCA incrémental

Étant donné la dimension élevée des images (40 000 variables par image après réduction et passage en niveaux de gris), nous appliquons un PCA (Analyse en Composantes Principales) incrémental avec des batchs de 512 images. Le nombre de composantes retenu est 256.
Nous profitons du parcours de toutes les images en batch pour normaliser les valeurs de chaque pixel (initialement des entiers de 0 à 255) en float de 0 à 1 (en float32, 2x moins gourmand en mémoire que les float64).

Justification :

- Le PCA permet de réduire drastiquement la dimensionnalité, facilitant l’apprentissage tout en éliminant la redondance d’information.

- La version incrémentale du PCA est adaptée au traitement de grandes bases de données, car elle évite de charger l’ensemble des images en mémoire simultanément.

- Le choix de 256 composantes vise un compromis entre compression et préservation de l’information utile.

- La normalisation adapte la donnée aux modèles souvent sensibles à l'échelle des variables, comme le PCA. Elle permet une convergence plus rapide et stabilise les calculs sans aucune perte d'information.

Conséquences :

- Réduction du nombre de variables de 40 000 à 256, soit un facteur de compression considérable.

- Perte d’une partie de l’information : seules les directions de variance les plus importantes sont conservées. Cela peut entraîner une diminution de la performance si des détails subtils mais discriminants sont contenus dans les dimensions supprimées.

- En contrepartie, les données obtenues sont beaucoup plus légères et exploitables efficacement par les modèles d’apprentissage. Les ressources requises ne nous permettent pas de nous en passer pour un jeu si grand.

- La normalisation alourdit la mémoire allouée aux données. (de int à float)

### Conclusion

Le preprocessing mis en place repose sur une logique de réduction de la complexité et de cohérence méthodologique avec les travaux textuels. Chaque étape a été pensée pour :

- Limiter les coûts computationnels (réduction de taille, passage en gris, PCA).

- Aligner les données images avec les données textuelles (équilibrage des classes, encodage et découpage identiques).

- Préserver autant que possible la qualité de l’information, malgré les pertes inévitables dues à la compression et à la suppression de la couleur.

Ces choix constituent un compromis entre faisabilité technique et qualité de l’information conservée.

### Alternative envisageable

Nous aurions pu effectuer une réduction d'information par image plus faible (conserver les couleurs, conserver plus de pixels ou un nombre de composantes du PCA plus grand) mais nous aurions dû drastiquement diminuer la taille du jeu de données en un échantillon drastiquement plus petit. La réduction à chaque étape étant très considérable, la taille de l'échantillon serait sûrement minime.

Nous pourrions toutefois réfléchir à : 
- Conserver les couleurs
- Diminuer la dimension de l'image pour compenser le traitement.
- Il faudrait comparer la pertinence de la conservation de l'information par PCA entre de 1. de nombreuses informations en gris ou 2. moins d'information mais colorées.